In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import glob
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision.models.segmentation.fcn import fcn_resnet50
from osgeo import gdal, ogr
from torch.utils.data import DataLoader, Dataset, random_split
import matplotlib.pyplot as plt

In [3]:
class CustomDataset(Dataset):
  '''
  自定义数据集类
  '''
  def __init__(self,images_dir, txt_path, transform=None):
    self.images_dir = images_dir
    self.imagelist = self.get_imagelist(txt_path)
    self.transform = transform

  def get_imagelist(self,txt_path):
    with open(txt_path,'r') as f:
      imagelist = f.readlines()
    return imagelist

  def __len__(self):
    # 返回数据集的大小
    return len(self.imagelist)

  def __getitem__(self,index):
    # 获取单个图片
    path = os.path.join(self.images_dir,self.imagelist[index][:-1])
    raster_file = gdal.Open(path)
    raster_array = raster_file.ReadAsArray()
    img_data = raster_array[0:3] # 前两个波段
    mask_data = raster_array[3] # label波段
    img = img_data.transpose(1,2,0) # 维度调整以适应神经网络输入
    img = np.nan_to_num(img)  # 去除空值
    img[0:2] = (img[0:2] + 50) / 51 # S1归一化，根据GEE信息，S1最大值为1，最小值为-50;
    img[2] = (img[2] - np.min(img[2])) / (np.max(img[2])-np.min(img[2])) # DEM归一化
    img[2][np.isnan(img[2])] = 0
    img[2][np.isinf(img[2])] = 0
    mask = np.zeros([512,512])
    mask[mask_data == 0] = 0
    mask[mask_data == 1] = 0
    mask[mask_data == 2] = 1
    mask[mask_data == 3] = 1
    if self.transform:
      image = self.transform(img)
      mask = self.transform(mask)
    return image,mask

  def get_image_names(self, idx):
    return self.imagelist[idx]

In [4]:
# 数据强化
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize([-3.71, -4.72], [-6.46, -8.15])
])

# 创建数据集
img_path = '/content/drive/MyDrive/RS/HAND/S1_split/'
train_path = '/content/drive/MyDrive/RS/HAND/train.txt'
test_path = '/content/drive/MyDrive/RS/HAND/test.txt'
train_dataset = CustomDataset(img_path, train_path , transform=transform)
test_dataset = CustomDataset(img_path, test_path, transform=transform)

# 创建数据集加载器
train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=4,shuffle=False)

In [5]:
# 定义块
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1,
                      stride=1, padding_mode='reflect', bias=False),

            nn.BatchNorm2d(out_channels),
            # nn.InstanceNorm2d(out_channels),  # nnUNet

            nn.ReLU(inplace=True),
            # nn.LeakyReLU(),  # nnUNet

            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1,
                      stride=1, padding_mode='reflect', bias=False),

            nn.BatchNorm2d(out_channels),
            # nn.InstanceNorm2d(out_channels),  # nnUNet

            nn.ReLU(inplace=True)
            # nn.LeakyReLU()  # nnUNet
        )

    def forward(self, x):
        return self.block(x)


class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpSample, self).__init__()
        self.block = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return self.block(x)


In [6]:
# 定义UNet模型
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=2):
        super().__init__()
        self.conv1 = DoubleConv(in_channels, 64)
        self.down1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = DoubleConv(64, 128)
        self.down2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = DoubleConv(128, 256)
        self.down3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4 = DoubleConv(256, 512)
        self.down4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv_mid = DoubleConv(512, 1024)

        self.up1 = UpSample(1024, 512)
        self.conv5 = DoubleConv(1024,512)
        self.up2 = UpSample(512, 256)
        self.conv6 = DoubleConv(512,256)
        self.up3 = UpSample(256, 128)
        self.conv7 = DoubleConv(256,128)
        self.up4 = UpSample(128, 64)
        self.conv8 = DoubleConv(128,64)

        self.out_channel = nn.Conv2d(64, out_channels, kernel_size=1, stride=1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        conv1 = self.conv1(x)
        down1 = self.down1(conv1)
        conv2 = self.conv2(down1)
        down2 = self.down2(conv2)
        conv3 = self.conv3(down2)
        down3 = self.down3(conv3)
        conv4 = self.conv4(down3)
        down4 = self.down4(conv4)

        conv_mid = self.conv_mid(down4)

        up1 = self.up1(conv_mid)
        cat1 = torch.cat([up1, conv4], dim=1)
        down5  = self.conv5(cat1)

        up2 = self.up2(down5)
        cat2 = torch.cat([up2, conv3], dim=1)
        down6  = self.conv6(cat2)

        up3 = self.up3(down6)
        cat3 = torch.cat([up3, conv2], dim=1)
        down7  = self.conv7(cat3)

        up4 = self.up4(down7)
        cat4 = torch.cat([up4, conv1], dim=1)
        down8  = self.conv8(cat4)

        out_channel = self.out_channel(down8)

        return out_channel

In [7]:
model = UNet()
# 定义损失函数、优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [8]:
def computeIOU(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()

  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  intersection = torch.sum(output * target)
  union = torch.sum(target) + torch.sum(output) - intersection
  iou = (intersection + .0000001) / (union + .0000001)

  if iou != iou:
    print("failed, replacing with 0")
    iou = torch.tensor(0).float()

  return iou

def computeAccuracy(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()

  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output.eq(target))

  return correct.float() / len(target)

def truePositives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output * target)

  return correct

def trueNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 0)
  correct = torch.sum(output * target)

  return correct

def falsePositives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 1)
  target = (target == 0)
  correct = torch.sum(output * target)

  return correct

def falseNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 1)
  correct = torch.sum(output * target)

  return correct

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 把网络设置在训练模式
model.train()
model.to(device)

cuda:0


UNet(
  (conv1): DoubleConv(
    (block): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, padding_mode=reflect)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, padding_mode=reflect)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): DoubleConv(
    (block): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, padding_mode=reflect)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False, padding_mode=r

In [10]:
loss_history = [] #训练过程中的loss数据
num_epochs = 20
for epoch in range(num_epochs):
    for i,data in enumerate(train_loader,0):
        inputs, targets = data[0].to(device), data[1].to(device)
        #(0) 复位优化器的梯度
        optimizer.zero_grad()
        #(1) 前向计算
        y_pred = model(inputs)
        #(2) 计算loss
        targets = targets.squeeze(1)
        loss = loss_fn(y_pred, targets.long())
        #(3) 反向求导
        loss.backward()
        #(4) 反向迭代
        optimizer.step()
        # 记录训练过程中的损失

    loss_history.append(loss.item())
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

<ipython-input-3-18e0d9dd406c>:29: RuntimeWarning: invalid value encountered in divide
  img[2] = (img[2] - np.min(img[2])) / (np.max(img[2])-np.min(img[2])) # DEM归一化


Epoch 1/20, Loss: 0.19254142045974731
Epoch 2/20, Loss: 0.24795499444007874
Epoch 3/20, Loss: 0.1457730084657669
Epoch 4/20, Loss: 0.2074679136276245
Epoch 5/20, Loss: 0.14553694427013397
Epoch 6/20, Loss: 0.5380618572235107
Epoch 7/20, Loss: 0.26109665632247925
Epoch 8/20, Loss: 1.1150954961776733
Epoch 9/20, Loss: 0.12217817455530167
Epoch 10/20, Loss: 0.1322135478258133
Epoch 11/20, Loss: 0.14549760520458221
Epoch 12/20, Loss: 0.12479268014431
Epoch 13/20, Loss: 0.21341374516487122
Epoch 14/20, Loss: 0.07713279128074646
Epoch 15/20, Loss: 0.07002125680446625
Epoch 16/20, Loss: 0.2605034112930298
Epoch 17/20, Loss: 0.10018409788608551
Epoch 18/20, Loss: 0.16497960686683655
Epoch 19/20, Loss: 0.05923672392964363
Epoch 20/20, Loss: 0.09923161566257477


In [11]:
model.eval()
iou = []
acc = []
TP = 0
TN = 0
FP = 0
FN = 0
for idx in range(len(test_dataset)):
  inputs, labels = test_dataset[idx]
  inputs = inputs.unsqueeze(0)
  inputs = inputs.to(device)
  labels = labels.to(device)
  with torch.no_grad():
    outputs = model(inputs)
  test_acc = computeAccuracy(outputs,labels).cpu().numpy()
  acc.append(test_acc)
  test_iou = computeIOU(outputs,labels).cpu().numpy()
  iou.append(test_iou)

  TP += truePositives(outputs,labels).cpu().numpy()
  TN += trueNegatives(outputs,labels).cpu().numpy()
  FP += falsePositives(outputs,labels).cpu().numpy()
  FN += falseNegatives(outputs,labels).cpu().numpy()

mean_iou = sum(iou) / len(iou)
mean_acc = sum(acc) / len(acc)

In [12]:
print(mean_acc)
print(mean_iou)

print(TP)
print(TN)
print(FP)
print(FN)

confusion_matrix = [TP,TN,FP,FN]

0.9579851081572384
0.3495338124471771
1140476.0
19703318
294862
619296


In [13]:
# 保存模型参数为字典
torch.save(model.state_dict(), '/content/drive/MyDrive/RS/HAND/UNet_S1D_state_dict.pth')
np.savetxt('/content/drive/MyDrive/RS/HAND/S1D_evaluate/S1D_UNet_loss.txt', loss_history)
np.savetxt('/content/drive/MyDrive/RS/HAND/S1D_evaluate/S1D_UNet_acc.txt', acc)
np.savetxt('/content/drive/MyDrive/RS/HAND/S1D_evaluate/S1D_UNet_iou.txt', iou)
np.savetxt('/content/drive/MyDrive/RS/HAND/S1D_evaluate/S1D_UNet_confusion_matrix.txt', confusion_matrix)